In [6]:
import pandas as pd
df=pd.read_csv('E:/Project/Project files/Microsoft/Data/cleaned_df.csv')
df.head()

,Unnamed: 0,AlertTitle,Category,IncidentGrade,EntityType,EvidenceRole,Sha256,IpAddress,Url,AccountUpn,...,FileName,FolderPath,OSFamily,OSVersion,CountryCode,State,City,Year,Month,Day
0,0,813,Malware,BenignPositive,File,Related,1296,360606,160396,673934,...,4132,4227,5,66,242,1445,10630,2024,6,6
1,1,25,Impact,BenignPositive,Machine,Impacted,138268,360606,160396,673934,...,289573,117668,5,66,242,1445,10630,2024,6,3
2,2,444,Execution,BenignPositive,Machine,Impacted,138268,360606,160396,673934,...,289573,117668,5,66,242,1445,10630,2024,6,13
3,3,1233,SuspiciousActivity,TruePositive,File,Related,138268,360606,160396,673934,...,97007,9177,5,66,242,1445,10630,2024,6,8
4,4,58829,Exfiltration,BenignPositive,File,Impacted,138268,360606,160396,673934,...,39978,114449,5,66,242,1445,10630,2024,6,9


In [7]:
df.drop(columns="Unnamed: 0",axis=1,inplace=True)

In [8]:
df.isnull().sum()

AlertTitle               0
Category                 0
IncidentGrade        24824
EntityType               0
EvidenceRole             0
Sha256                   0
IpAddress                0
Url                      0
AccountUpn               0
AccountName              0
DeviceName               0
RegistryKey              0
RegistryValueName        0
RegistryValueData        0
ApplicationName          0
FileName                 0
FolderPath               0
OSFamily                 0
OSVersion                0
CountryCode              0
State                    0
City                     0
Year                     0
Month                    0
Day                      0
dtype: int64

In [9]:
df.dropna(inplace=True)

In [10]:
df.isnull().sum()

AlertTitle           0
Category             0
IncidentGrade        0
EntityType           0
EvidenceRole         0
Sha256               0
IpAddress            0
Url                  0
AccountUpn           0
AccountName          0
DeviceName           0
RegistryKey          0
RegistryValueName    0
RegistryValueData    0
ApplicationName      0
FileName             0
FolderPath           0
OSFamily             0
OSVersion            0
CountryCode          0
State                0
City                 0
Year                 0
Month                0
Day                  0
dtype: int64

In [11]:
df['IncidentGrade'].unique()

array(['BenignPositive', 'TruePositive', 'FalsePositive'], dtype=object)

In [12]:
# Sampling from each category
BenignPositive = df[df['IncidentGrade'] == "BenignPositive"].sample(500000, random_state=42)
TruePositive = df[df['IncidentGrade'] == "TruePositive"].sample(500000, random_state=42)
FalsePositive = df[df['IncidentGrade'] == "FalsePositive"].sample(500000, random_state=42)

# Concatenating them into one DataFrame
df_sampled = pd.concat([BenignPositive, TruePositive, FalsePositive], axis=0)

# Reset index
df_sampled = df_sampled.reset_index(drop=True)

# Display the shape of the new DataFrame
df_sampled.head()

,AlertTitle,Category,IncidentGrade,EntityType,EvidenceRole,Sha256,IpAddress,Url,AccountUpn,AccountName,...,FileName,FolderPath,OSFamily,OSVersion,CountryCode,State,City,Year,Month,Day
0,13,Exfiltration,BenignPositive,File,Impacted,138268,360606,160396,673934,453297,...,60323,58058,5,66,242,1445,10630,2024,6,5
1,54970,Exfiltration,BenignPositive,MailMessage,Impacted,138268,360606,160396,330666,453297,...,289573,117668,5,66,242,1445,10630,2024,6,3
2,10274,Exfiltration,BenignPositive,File,Impacted,1746,360606,160396,673934,453297,...,1991,708,5,66,242,1445,10630,2024,6,3
3,22,InitialAccess,BenignPositive,Mailbox,Impacted,138268,360606,160396,80363,80948,...,289573,117668,5,66,242,1445,10630,2024,6,11
4,492,CredentialAccess,BenignPositive,Ip,Related,138268,216810,160396,673934,453297,...,289573,117668,5,66,242,1445,10630,2024,5,30


In [13]:
df_sampled.to_csv("sampled.csv",index=False)

In [14]:
# encode
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
for col in df_sampled.columns:
    df_sampled[col]=encoder.fit_transform(df_sampled[col])
print("Done")

import pickle
with open("encoders.pkl",'wb')as f:
    pickle.dump(encoder,f)

Done


In [15]:
from sklearn.model_selection import train_test_split

X = df_sampled.drop(columns=['IncidentGrade'])  # Features
y = df_sampled['IncidentGrade']  # Target
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,test_size=0.25)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(1125000, 24) (375000, 24) (1125000,) (375000,)


In [16]:
from imblearn.over_sampling import RandomOverSampler

# Apply Random Oversampling (ROS)
ros = RandomOverSampler(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

df_resampled = pd.DataFrame(X_resampled, columns=X.columns)
df_resampled['IncidentGrade'] = y_resampled

# Check the new class distribution
print(df_resampled['IncidentGrade'].value_counts())


IncidentGrade
2    375639
0    375639
1    375639
Name: count, dtype: int64


In [17]:
X_train = df_resampled.drop(columns="IncidentGrade",axis=1)
y_train = df_resampled['IncidentGrade']

# Model selection

In [18]:
#models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

#Evaluation metrics
from sklearn.metrics import accuracy_score,classification_report

In [ ]:
model=DecisionTreeClassifier(max_depth=)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.8192053333333333


In [20]:
model=XGBClassifier()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.7436666666666667


In [21]:
from sklearn.model_selection import GridSearchCV
dt = DecisionTreeClassifier()

# Define the parameter grid
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Apply GridSearchCV
grid_search = GridSearchCV(estimator=dt, param_grid=param_grid,cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best parameters
print("Best Parameters:", grid_search.best_params_)

# Get the best score
print("Best Cross-Validation Accuracy:", grid_search.best_score_)

# Evaluate on the test set
best_model = grid_search.best_estimator_
test_accuracy = best_model.score(X_test, y_test)
print("Test Set Accuracy:", test_accuracy)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best Parameters: {'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2}
Best Cross-Validation Accuracy: 0.8146287613831724
Test Set Accuracy: 0.820832


In [23]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.67      0.81      0.73    124361
           1       0.76      0.68      0.72    125273
           2       0.83      0.73      0.78    125366

    accuracy                           0.74    375000
   macro avg       0.75      0.74      0.74    375000
weighted avg       0.75      0.74      0.74    375000



In [24]:
import pickle 
with open("model.pkl",'wb') as f:
    pickle.dump(best_model,f)
print("Done")

Done
